In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import bs4 as bs
import urllib.request

### Extracting features of the 2024 Movies from the Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2024"

In [3]:
sauce = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(sauce , 'lxml')

In [4]:
tables = soup.find_all('table' , class_='wikitable sortable')

In [5]:
df1 = pd.read_html(str(tables[1]))[0]
df2 = pd.read_html(str(tables[2]))[0]
df3 = pd.read_html(str(tables[3]))[0]
df4 = pd.read_html(str(tables[4]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [6]:
df = df1.append(df2.append(df3.append(df4 , ignore_index=True) , ignore_index=True) , ignore_index=True)

In [7]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,2.0,The Mummy Murders,Gravitas Ventures,Colin Bressler (director/screenplay); Will Don...,[3]
1,JANUARY,3.0,Self Reliance,Neon / Hulu / MRC / Paramount Global Content D...,Jake Johnson (director/screenplay); Jake Johns...,[4]
2,JANUARY,4.0,DarkGame,Gravitas Ventures,"Howard J. Ford (director); Gary Grant, Niall J...",[5]
3,JANUARY,5.0,Night Swim,Universal Pictures / Blumhouse Productions / A...,Bryce McGuire (director/screenplay); Wyatt Rus...,[6]
4,JANUARY,5.0,He Went That Way,Vertical Entertainment / Mister Smith Entertai...,Jeffrey Darling (director); Evan M. Wiener (sc...,[7]
...,...,...,...,...,...,...
398,DECEMBER,20.0,Babygirl,A24 / 2AM,Halina Reijn (director/screenplay); Nicole Kid...,[313]
399,DECEMBER,20.0,Homestead,Angel Studios,"Ben Smallbone (director); Phillip Abraham, Lea...",[379]
400,DECEMBER,25.0,Nosferatu,Focus Features / Studio 8,Robert Eggers (director/screenplay); Bill Skar...,[380]
401,DECEMBER,25.0,A Complete Unknown,Searchlight Pictures / The Picture Company,James Mangold (director/screenplay); Jay Cocks...,[381]


In [8]:
df_2024 = df[['Title' , 'Cast and crew']]

In [9]:
df_2024

,Title,Cast and crew
0,The Mummy Murders,Colin Bressler (director/screenplay); Will Don...
1,Self Reliance,Jake Johnson (director/screenplay); Jake Johns...
2,DarkGame,"Howard J. Ford (director); Gary Grant, Niall J..."
3,Night Swim,Bryce McGuire (director/screenplay); Wyatt Rus...
4,He Went That Way,Jeffrey Darling (director); Evan M. Wiener (sc...
...,...,...
398,Babygirl,Halina Reijn (director/screenplay); Nicole Kid...
399,Homestead,"Ben Smallbone (director); Phillip Abraham, Lea..."
400,Nosferatu,Robert Eggers (director/screenplay); Bill Skar...
401,A Complete Unknown,James Mangold (director/screenplay); Jay Cocks...


In [10]:
df_2024.isnull().sum()

Title            1
Cast and crew    1
dtype: int64

In [11]:
df_2024 = df_2024.dropna(how='any')

In [12]:
df_2024.isnull().sum()

Title            0
Cast and crew    0
dtype: int64

In [13]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'fce0af3409e6113c9b3c75aaf49341bb'

In [14]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    try:
        result = tmdb_movie.search(x)
        print(f"Search result for '{x}': {result}")  # Debug print
        if not result:
            print("No results found.")
            return np.NaN
        else:
            movie = result[0]
            print(f"First movie in result: {movie}")  # Debug print
            movie_id = getattr(movie, 'id', None)
            if movie_id:
                response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb.api_key}')
                data_json = response.json()
                print(f"API response for movie ID {movie_id}: {data_json}")  # Debug print
                if 'genres' in data_json and data_json['genres']:
                    genre_str = " "
                    for genre in data_json['genres']:
                        genres.append(genre['name'])
                    return genre_str.join(genres)
                else:
                    print("No genres found in movie data.")
                    return np.NaN
            else:
                print("No movie ID found.")
                return np.NaN
    except Exception as e:
        print(f"An error occurred: {e}")
        return np.NaN

# Assuming df_2024 is your DataFrame and 'Title' is the column name
df_2024['genres'] = df_2024['Title'].map(lambda x: get_genre(str(x)))

Search result for 'The Mummy Murders': {'page': 1, 'results': [{'adult': False, 'backdrop_path': None, 'genre_ids': [27, 80], 'id': 1221596, 'original_language': 'en', 'original_title': 'The Mummy Murders', 'overview': "When an innocent man is charged with a serial killer's crimes, the real killer dubbed “The Mummy Murderer”, decides to share his story with Alexis, a hungry reporter for the local paper. She finds herself in the middle of his twisted game and is forced to make a choice between saving a life or propelling her career to the next level.", 'popularity': 0.452, 'poster_path': '/jlBX5ay7e2Oc6eEax6sNWXaus7l.jpg', 'release_date': '2024-01-01', 'title': 'The Mummy Murders', 'video': False, 'vote_average': 0.0, 'vote_count': 0}, {'adult': False, 'backdrop_path': '/wXe7yoFV4EKz6x0RmqUigXW3ohR.jpg', 'genre_ids': [80, 99], 'id': 1225861, 'original_language': 'en', 'original_title': "Mummy's Little Murderer: The Killing of Emily Longley", 'overview': 'On May 7th, 2011, 20-year-old El

Search result for 'Night Swim': {'page': 1, 'results': [{'adult': False, 'backdrop_path': '/aZ8dBIvpDFp9cp23MfBiY5mWfuy.jpg', 'genre_ids': [27], 'id': 1072342, 'original_language': 'en', 'original_title': 'Night Swim', 'overview': "Forced into early retirement by a degenerative illness, former baseball player Ray Waller moves into a new house with his wife and two children. He hopes that the backyard swimming pool will be fun for the kids and provide physical therapy for himself. However, a dark secret from the home's past soon unleashes a malevolent force that drags the family into the depths of inescapable terror.", 'popularity': 50.864, 'poster_path': '/gSkfBGdxdialBMM7P02V4hcI6Ij.jpg', 'release_date': '2024-01-03', 'title': 'Night Swim', 'video': False, 'vote_average': 5.64, 'vote_count': 500}, {'adult': False, 'backdrop_path': None, 'genre_ids': [27], 'id': 305966, 'original_language': 'en', 'original_title': 'Night Swim', 'overview': 'A woman swimming in her pool at night is watc

Search result for 'The Painter': {'page': 1, 'results': [{'adult': False, 'backdrop_path': '/6OnoMgGFuZ921eV8v8yEyXoag19.jpg', 'genre_ids': [28, 53], 'id': 1211957, 'original_language': 'en', 'original_title': 'The Painter', 'overview': 'An ex-CIA operative is thrown back into a dangerous world when a mysterious woman from his past resurfaces. Now exposed and targeted by a relentless killer and a rogue black ops program, he must rely on skills he thought he left behind in a high-stakes game of survival.', 'popularity': 45.548, 'poster_path': '/UZ0ydgbXtnrq8xZCI5lHVXVcH9.jpg', 'release_date': '2024-01-05', 'title': 'The Painter', 'video': False, 'vote_average': 6.304, 'vote_count': 107}, {'adult': False, 'backdrop_path': '/dbKn2XMpuZawtkmW5TkU3yW7cqf.jpg', 'genre_ids': [18, 14], 'id': 1282913, 'original_language': 'en', 'original_title': 'The Painter', 'overview': 'A teen girl, trapped by the normalised pressures of academia, decides to rebel in order to pursue her passion through artis

API response for movie ID 1211957: {'adult': False, 'backdrop_path': '/6OnoMgGFuZ921eV8v8yEyXoag19.jpg', 'belongs_to_collection': None, 'budget': 0, 'genres': [{'id': 28, 'name': 'Action'}, {'id': 53, 'name': 'Thriller'}], 'homepage': '', 'id': 1211957, 'imdb_id': 'tt30253266', 'origin_country': ['US'], 'original_language': 'en', 'original_title': 'The Painter', 'overview': 'An ex-CIA operative is thrown back into a dangerous world when a mysterious woman from his past resurfaces. Now exposed and targeted by a relentless killer and a rogue black ops program, he must rely on skills he thought he left behind in a high-stakes game of survival.', 'popularity': 45.548, 'poster_path': '/UZ0ydgbXtnrq8xZCI5lHVXVcH9.jpg', 'production_companies': [{'id': 177095, 'logo_path': '/nfwWCvMNaK687bjyM7un8p7ilz0.png', 'name': 'SP Media Group', 'origin_country': 'US'}], 'production_countries': [{'iso_3166_1': 'US', 'name': 'United States of America'}], 'release_date': '2024-01-05', 'revenue': 0, 'runtime

API response for movie ID 912600: {'adult': False, 'backdrop_path': '/qTNBCN5XQhFy2lgH0XTMg87AI4N.jpg', 'belongs_to_collection': None, 'budget': 0, 'genres': [{'id': 53, 'name': 'Thriller'}, {'id': 18, 'name': 'Drama'}], 'homepage': '', 'id': 912600, 'imdb_id': 'tt15091524', 'origin_country': ['US'], 'original_language': 'en', 'original_title': 'Some Other Woman', 'overview': 'Eve was promised a few months on a tropical island for her husband’s work... but years later, unable to build the family they wanted, she has given up on any of her own desires. As island fever intensifies, reality unravels around Eve as a strange woman begins taking over her life piece by piece.', 'popularity': 8.138, 'poster_path': '/kiXZm4lEuE5DbvUssNFi10X3J4J.jpg', 'production_companies': [{'id': 41608, 'logo_path': '/vhSODaSCgrzG5ofUc0xTln9AMYZ.png', 'name': 'Productivity Media', 'origin_country': 'CA'}, {'id': 193159, 'logo_path': '/9qDnUCKcn6WOJmD44EmHg0sxdUB.png', 'name': 'Balcony 9 Productions', 'origin_

...

Search result for 'Sonic the Hedgehog 3': {'page': 1, 'results': [{'adult': False, 'backdrop_path': '/qYXeKnlqDkFliZ1FVdjGdg8mpNY.jpg', 'genre_ids': [878, 35, 10751, 28], 'id': 939243, 'original_language': 'en', 'original_title': 'Sonic the Hedgehog 3', 'overview': 'Sonic, Tails, and Knuckles face a mysterious new adversary, Shadow the Hedgehog, as Dr. Robotnik reemerges after his defeat with a new plan.', 'popularity': 95.612, 'poster_path': '/hknd8HZkMm7QZAsA94YJvIuXqh9.jpg', 'release_date': '2024-12-19', 'title': 'Sonic the Hedgehog 3', 'video': False, 'vote_average': 0.0, 'vote_count': 0}], 'total_pages': 1, 'total_results': 1}
First movie in result: {'adult': False, 'backdrop_path': '/qYXeKnlqDkFliZ1FVdjGdg8mpNY.jpg', 'genre_ids': [878, 35, 10751, 28], 'id': 939243, 'original_language': 'en', 'original_title': 'Sonic the Hedgehog 3', 'overview': 'Sonic, Tails, and Knuckles face a mysterious new adversary, Shadow the Hedgehog, as Dr. Robotnik reemerges after his defeat with a new p

Search result for 'Homestead': {'page': 1, 'results': [{'adult': False, 'backdrop_path': '/dzQsXr8rpA0cuK7h1R0LKvczJS1.jpg', 'genre_ids': [27, 53, 37], 'id': 1015957, 'original_language': 'en', 'original_title': 'Homestead', 'overview': "A family of homesteaders taken captive by a gang of outlaws. Their survival comes to rest in the hands of Irene: a loud-mouthed 12-year-old girl who's got an uncanny knack for shooting guns.", 'popularity': 14.957, 'poster_path': '/aibubaLT6ihRtKEraGBDU82w8aA.jpg', 'release_date': '2023-03-03', 'title': 'Homestead', 'video': False, 'vote_average': 5.0, 'vote_count': 10}, {'adult': False, 'backdrop_path': None, 'genre_ids': [18, 878, 53], 'id': 915527, 'original_language': 'en', 'original_title': 'Homestead', 'overview': '', 'popularity': 2.001, 'poster_path': None, 'release_date': '', 'title': 'Homestead', 'video': False, 'vote_average': 0.0, 'vote_count': 0}, {'adult': False, 'backdrop_path': None, 'genre_ids': [28, 18, 53], 'id': 1217690, 'original_l

API response for movie ID 426063: {'adult': False, 'backdrop_path': '/ArM4yzl1oxCrJVUngw5lKAEpkb7.jpg', 'belongs_to_collection': None, 'budget': 0, 'genres': [{'id': 18, 'name': 'Drama'}, {'id': 14, 'name': 'Fantasy'}, {'id': 27, 'name': 'Horror'}], 'homepage': 'https://www.focusfeatures.com/nosferatu', 'id': 426063, 'imdb_id': 'tt5040012', 'origin_country': ['US'], 'original_language': 'en', 'original_title': 'Nosferatu', 'overview': 'A gothic tale of obsession between a haunted young woman and the terrifying vampire infatuated with her, causing untold horror in its wake.', 'popularity': 21.884, 'poster_path': '/gVjHF375pdFANPMXUT3aJxoOi0J.jpg', 'production_companies': [{'id': 77863, 'logo_path': '/7z6YOzvdk70YY8OaRbn5mfxrvbO.png', 'name': 'Studio 8', 'origin_country': 'US'}, {'id': 10146, 'logo_path': '/xnFIOeq5cKw09kCWqV7foWDe4AA.png', 'name': 'Focus Features', 'origin_country': 'US'}, {'id': 41641, 'logo_path': None, 'name': 'Maiden Voyage Pictures', 'origin_country': 'GB'}], 'prod

API response for movie ID 610219: {'adult': False, 'backdrop_path': '/6TKs48HU6vt8EOx8Xh2Th9ZgpUH.jpg', 'belongs_to_collection': None, 'budget': 0, 'genres': [{'id': 18, 'name': 'Drama'}], 'homepage': '', 'id': 610219, 'imdb_id': 'tt6133444', 'origin_country': ['US'], 'original_language': 'en', 'original_title': 'The Fire Inside', 'overview': "The story of Claressa 'T-Rex' Shields, a boxer from Flint, Michigan who trained to become the first woman in her country's history to win an Olympic gold medal in the sport.", 'popularity': 7.564, 'poster_path': '/hpPOM6NVZKsF1DPqb58BiE3GnsE.jpg', 'production_companies': [{'id': 21, 'logo_path': '/usUnaYV6hQnlVAXP6r4HwrlLFPG.png', 'name': 'Metro-Goldwyn-Mayer', 'origin_country': 'US'}, {'id': 11370, 'logo_path': None, 'name': 'Michael De Luca Productions', 'origin_country': 'US'}, {'id': 103697, 'logo_path': '/9gsomzlpj1lLR7kupysQ7nSFcHh.png', 'name': 'PASTEL', 'origin_country': 'US'}], 'production_countries': [{'iso_3166_1': 'US', 'name': 'Unite

c:\users\paras rana\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [15]:
df_2024

,Title,Cast and crew,genres
0,The Mummy Murders,Colin Bressler (director/screenplay); Will Don...,Horror Crime
1,Self Reliance,Jake Johnson (director/screenplay); Jake Johns...,Comedy Thriller
2,DarkGame,"Howard J. Ford (director); Gary Grant, Niall J...",Horror Thriller
3,Night Swim,Bryce McGuire (director/screenplay); Wyatt Rus...,Horror
4,He Went That Way,Jeffrey Darling (director); Evan M. Wiener (sc...,Mystery Thriller Crime Drama
...,...,...,...
398,Babygirl,Halina Reijn (director/screenplay); Nicole Kid...,Thriller
399,Homestead,"Ben Smallbone (director); Phillip Abraham, Lea...",Horror Thriller Western
400,Nosferatu,Robert Eggers (director/screenplay); Bill Skar...,Drama Fantasy Horror
401,A Complete Unknown,James Mangold (director/screenplay); Jay Cocks...,Drama Music History


In [16]:
df_2024.isnull().sum()

Title            0
Cast and crew    0
genres           2
dtype: int64

In [17]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [18]:
df_2024['director_name'] = df_2024['Cast and crew'].map(lambda x: get_director(x))

c:\users\paras rana\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
df_2024['director_name']

0       Colin Bressler
1         Jake Johnson
2       Howard J. Ford
3        Bryce McGuire
4      Jeffrey Darling
            ...       
398       Halina Reijn
399      Ben Smallbone
400      Robert Eggers
401      James Mangold
402    Rachel Morrison
Name: director_name, Length: 402, dtype: object

In [20]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [21]:
df_2024['actor_1_name'] = df_2024['Cast and crew'].map(lambda x: get_actor1(x))

c:\users\paras rana\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
df_2024['actor_1_name']

0      Leila Annastasia Scott
1                Jake Johnson
2                 Ed Westwick
3               Wyatt Russell
4                Jacob Elordi
                ...          
398             Nicole Kidman
399            Neal McDonough
400            Bill Skarsgård
401         Timothée Chalamet
402              Ryan Destiny
Name: actor_1_name, Length: 402, dtype: object

In [23]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [24]:
df_2024['actor_2_name'] = df_2024['Cast and crew'].apply(lambda x: get_actor2(x))

c:\users\paras rana\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
df_2024['actor_2_name']

0       Jason Scarbrough
1          Anna Kendrick
2      Andrew P. Stephen
3           Kerry Condon
4         Zachary Quinto
             ...        
398     Harris Dickinson
399        Dawn Olivieri
400       Nicholas Hoult
401        Edward Norton
402    Brian Tyree Henry
Name: actor_2_name, Length: 402, dtype: object

In [26]:
import numpy as np

def get_actor3(x):
    # Check if the input is a string
    if isinstance(x, str):
        # Split the string after "screenplay); " and then by ", "
        parts = (x.split("screenplay); ")[-1]).split(", ")
        
        # Check if there are at least 3 elements after the split
        if len(parts) < 3:
            return np.NaN
        else:
            return parts[2]
    else:
        # If x is not a string, return NaN
        return np.NaN

In [27]:
df_2024['actor_3_name'] = df['Cast and crew'].map(lambda x: get_actor3(x))

c:\users\paras rana\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
df_2024['actor_3_name']

0          Jeff Caperton
1        Natalie Morales
2      Natalya Tsvetkova
3        Amélie Hoeferle
4       Patrick J. Adams
             ...        
398     Antonio Banderas
399        Currie Graham
400       Lily-Rose Depp
401         Elle Fanning
402           Judy Greer
Name: actor_3_name, Length: 402, dtype: object

In [29]:
df_2024

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Mummy Murders,Colin Bressler (director/screenplay); Will Don...,Horror Crime,Colin Bressler,Leila Annastasia Scott,Jason Scarbrough,Jeff Caperton
1,Self Reliance,Jake Johnson (director/screenplay); Jake Johns...,Comedy Thriller,Jake Johnson,Jake Johnson,Anna Kendrick,Natalie Morales
2,DarkGame,"Howard J. Ford (director); Gary Grant, Niall J...",Horror Thriller,Howard J. Ford,Ed Westwick,Andrew P. Stephen,Natalya Tsvetkova
3,Night Swim,Bryce McGuire (director/screenplay); Wyatt Rus...,Horror,Bryce McGuire,Wyatt Russell,Kerry Condon,Amélie Hoeferle
4,He Went That Way,Jeffrey Darling (director); Evan M. Wiener (sc...,Mystery Thriller Crime Drama,Jeffrey Darling,Jacob Elordi,Zachary Quinto,Patrick J. Adams
...,...,...,...,...,...,...,...
398,Babygirl,Halina Reijn (director/screenplay); Nicole Kid...,Thriller,Halina Reijn,Nicole Kidman,Harris Dickinson,Antonio Banderas
399,Homestead,"Ben Smallbone (director); Phillip Abraham, Lea...",Horror Thriller Western,Ben Smallbone,Neal McDonough,Dawn Olivieri,Currie Graham
400,Nosferatu,Robert Eggers (director/screenplay); Bill Skar...,Drama Fantasy Horror,Robert Eggers,Bill Skarsgård,Nicholas Hoult,Lily-Rose Depp
401,A Complete Unknown,James Mangold (director/screenplay); Jay Cocks...,Drama Music History,James Mangold,Timothée Chalamet,Edward Norton,Elle Fanning


In [30]:
df_2024 = df_2024.rename(columns={'Title' : 'movie_title'})

In [31]:
newdf_2024 = df_2024.loc[: , ['director_name' , 'actor_1_name' , 'actor_2_name' , 'actor_3_name' , 'genres' , 'movie_title']]

In [32]:
newdf_2024

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Colin Bressler,Leila Annastasia Scott,Jason Scarbrough,Jeff Caperton,Horror Crime,The Mummy Murders
1,Jake Johnson,Jake Johnson,Anna Kendrick,Natalie Morales,Comedy Thriller,Self Reliance
2,Howard J. Ford,Ed Westwick,Andrew P. Stephen,Natalya Tsvetkova,Horror Thriller,DarkGame
3,Bryce McGuire,Wyatt Russell,Kerry Condon,Amélie Hoeferle,Horror,Night Swim
4,Jeffrey Darling,Jacob Elordi,Zachary Quinto,Patrick J. Adams,Mystery Thriller Crime Drama,He Went That Way
...,...,...,...,...,...,...
398,Halina Reijn,Nicole Kidman,Harris Dickinson,Antonio Banderas,Thriller,Babygirl
399,Ben Smallbone,Neal McDonough,Dawn Olivieri,Currie Graham,Horror Thriller Western,Homestead
400,Robert Eggers,Bill Skarsgård,Nicholas Hoult,Lily-Rose Depp,Drama Fantasy Horror,Nosferatu
401,James Mangold,Timothée Chalamet,Edward Norton,Elle Fanning,Drama Music History,A Complete Unknown


In [33]:
newdf_2024['actor_1_name'] = newdf_2024['actor_1_name'].replace(np.nan , 'unknown')
newdf_2024['actor_2_name'] = newdf_2024['actor_2_name'].replace(np.nan , 'unknown')
newdf_2024['actor_3_name'] = newdf_2024['actor_3_name'].replace(np.nan , 'unknown')

In [34]:
newdf_2024['movie_title'] = newdf_2024['movie_title'].str.lower()

In [35]:
newdf_2024['comb'] = newdf_2024['actor_1_name'] + ' ' + newdf_2024['actor_2_name'] + ' ' + newdf_2024['actor_3_name'] + ' ' + newdf_2024['director_name'] + ' ' + newdf_2024['genres']

In [36]:
newdf_2024

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Colin Bressler,Leila Annastasia Scott,Jason Scarbrough,Jeff Caperton,Horror Crime,the mummy murders,Leila Annastasia Scott Jason Scarbrough Jeff C...
1,Jake Johnson,Jake Johnson,Anna Kendrick,Natalie Morales,Comedy Thriller,self reliance,Jake Johnson Anna Kendrick Natalie Morales Jak...
2,Howard J. Ford,Ed Westwick,Andrew P. Stephen,Natalya Tsvetkova,Horror Thriller,darkgame,Ed Westwick Andrew P. Stephen Natalya Tsvetkov...
3,Bryce McGuire,Wyatt Russell,Kerry Condon,Amélie Hoeferle,Horror,night swim,Wyatt Russell Kerry Condon Amélie Hoeferle Bry...
4,Jeffrey Darling,Jacob Elordi,Zachary Quinto,Patrick J. Adams,Mystery Thriller Crime Drama,he went that way,Jacob Elordi Zachary Quinto Patrick J. Adams J...
...,...,...,...,...,...,...,...
398,Halina Reijn,Nicole Kidman,Harris Dickinson,Antonio Banderas,Thriller,babygirl,Nicole Kidman Harris Dickinson Antonio Bandera...
399,Ben Smallbone,Neal McDonough,Dawn Olivieri,Currie Graham,Horror Thriller Western,homestead,Neal McDonough Dawn Olivieri Currie Graham Ben...
400,Robert Eggers,Bill Skarsgård,Nicholas Hoult,Lily-Rose Depp,Drama Fantasy Horror,nosferatu,Bill Skarsgård Nicholas Hoult Lily-Rose Depp R...
401,James Mangold,Timothée Chalamet,Edward Norton,Elle Fanning,Drama Music History,a complete unknown,Timothée Chalamet Edward Norton Elle Fanning J...


In [37]:
newdf_2024.isnull().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           2
movie_title      0
comb             2
dtype: int64

In [38]:
newdf_2024.dropna()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Colin Bressler,Leila Annastasia Scott,Jason Scarbrough,Jeff Caperton,Horror Crime,the mummy murders,Leila Annastasia Scott Jason Scarbrough Jeff C...
1,Jake Johnson,Jake Johnson,Anna Kendrick,Natalie Morales,Comedy Thriller,self reliance,Jake Johnson Anna Kendrick Natalie Morales Jak...
2,Howard J. Ford,Ed Westwick,Andrew P. Stephen,Natalya Tsvetkova,Horror Thriller,darkgame,Ed Westwick Andrew P. Stephen Natalya Tsvetkov...
3,Bryce McGuire,Wyatt Russell,Kerry Condon,Amélie Hoeferle,Horror,night swim,Wyatt Russell Kerry Condon Amélie Hoeferle Bry...
4,Jeffrey Darling,Jacob Elordi,Zachary Quinto,Patrick J. Adams,Mystery Thriller Crime Drama,he went that way,Jacob Elordi Zachary Quinto Patrick J. Adams J...
...,...,...,...,...,...,...,...
398,Halina Reijn,Nicole Kidman,Harris Dickinson,Antonio Banderas,Thriller,babygirl,Nicole Kidman Harris Dickinson Antonio Bandera...
399,Ben Smallbone,Neal McDonough,Dawn Olivieri,Currie Graham,Horror Thriller Western,homestead,Neal McDonough Dawn Olivieri Currie Graham Ben...
400,Robert Eggers,Bill Skarsgård,Nicholas Hoult,Lily-Rose Depp,Drama Fantasy Horror,nosferatu,Bill Skarsgård Nicholas Hoult Lily-Rose Depp R...
401,James Mangold,Timothée Chalamet,Edward Norton,Elle Fanning,Drama Music History,a complete unknown,Timothée Chalamet Edward Norton Elle Fanning J...


In [39]:
old_df = pd.read_csv("main_data.csv")

In [40]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: the force awakens,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
7101,Sam Taylor-Johnson,Dakota Johnson,Jamie Dornan,Jennifer Ehle,"Romance , Erotic Film , Melodrama , Thriller ,...",fifty shades darker,Dakota Johnson Jamie Dornan Jennifer Ehle Sam ...
7102,Sam Taylor-Johnson,Dakota Johnson,Jamie Dornan,Jennifer Ehle,"Romance , Erotic Film , Melodrama , Thriller ,...",fifty shades freed,Dakota Johnson Jamie Dornan Jennifer Ehle Sam ...
7103,Shawn Levy,Ryan Reynolds,Hugh Jackman,Emma Corrin,"Action , Adventure , Comedy , Sci-Fi , Superhero",deadpool & wolverine,Ryan Reynolds Hugh Jackman Emma Corrin Shawn L...
7104,S.J. Clarkson,Dakota Johnson,Sydney Sweeney,Isabela Merced,"Action , Adventure , Fantasy , Thriller , Sci-...",madame web,Dakota Johnson Sydney Sweeney Isabela Merced S...


In [41]:
old_df.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [42]:
final_df = old_df.append(newdf_2024 , ignore_index=True)

In [43]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: the force awakens,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
7503,Halina Reijn,Nicole Kidman,Harris Dickinson,Antonio Banderas,Thriller,babygirl,Nicole Kidman Harris Dickinson Antonio Bandera...
7504,Ben Smallbone,Neal McDonough,Dawn Olivieri,Currie Graham,Horror Thriller Western,homestead,Neal McDonough Dawn Olivieri Currie Graham Ben...
7505,Robert Eggers,Bill Skarsgård,Nicholas Hoult,Lily-Rose Depp,Drama Fantasy Horror,nosferatu,Bill Skarsgård Nicholas Hoult Lily-Rose Depp R...
7506,James Mangold,Timothée Chalamet,Edward Norton,Elle Fanning,Drama Music History,a complete unknown,Timothée Chalamet Edward Norton Elle Fanning J...


In [44]:
final_df.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           2
movie_title      0
comb             2
dtype: int64

In [45]:
final_df = final_df.dropna(how='any')

In [46]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: the force awakens,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
7503,Halina Reijn,Nicole Kidman,Harris Dickinson,Antonio Banderas,Thriller,babygirl,Nicole Kidman Harris Dickinson Antonio Bandera...
7504,Ben Smallbone,Neal McDonough,Dawn Olivieri,Currie Graham,Horror Thriller Western,homestead,Neal McDonough Dawn Olivieri Currie Graham Ben...
7505,Robert Eggers,Bill Skarsgård,Nicholas Hoult,Lily-Rose Depp,Drama Fantasy Horror,nosferatu,Bill Skarsgård Nicholas Hoult Lily-Rose Depp R...
7506,James Mangold,Timothée Chalamet,Edward Norton,Elle Fanning,Drama Music History,a complete unknown,Timothée Chalamet Edward Norton Elle Fanning J...


In [47]:
final_df1 = final_df.drop_duplicates()

In [48]:
final_df1

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: the force awakens,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
7503,Halina Reijn,Nicole Kidman,Harris Dickinson,Antonio Banderas,Thriller,babygirl,Nicole Kidman Harris Dickinson Antonio Bandera...
7504,Ben Smallbone,Neal McDonough,Dawn Olivieri,Currie Graham,Horror Thriller Western,homestead,Neal McDonough Dawn Olivieri Currie Graham Ben...
7505,Robert Eggers,Bill Skarsgård,Nicholas Hoult,Lily-Rose Depp,Drama Fantasy Horror,nosferatu,Bill Skarsgård Nicholas Hoult Lily-Rose Depp R...
7506,James Mangold,Timothée Chalamet,Edward Norton,Elle Fanning,Drama Music History,a complete unknown,Timothée Chalamet Edward Norton Elle Fanning J...


In [49]:
final_df1.to_csv('main_data1.csv' , index=False)